# Bölüm 1: Bazı Kararlar

In [3]:
import openpyxl
import pandas as pd

In [8]:
file_path = "C:/Users/belki/OneDrive - marmara.edu.tr/Masaüstü/Yüksek Lisans/Tez/Affective Polarization/TIDA Python.xlsx" #excel dosyası

pol = pd.read_excel(file_path)

print(pol.head()) #veri kontrolü

   q0001  q0002  q0003  q0004  q0005  q0006  q0007  q0008  q0009  q0010  ...  \
0      2      4     73      1    NaN      1    5.0      5      3      3  ...   
1      2      9     73      1    NaN      2    NaN      5      8      9  ...   
2      2      8     73      1    NaN      2    NaN      7      1      3  ...   
3      1     29     73      2    1.0      2    NaN      7      6      6  ...   
4      2     20     40      2    4.0      2    NaN      5      2      2  ...   

   dindarR  kirkent  nutstr  basortu  disortu  evortu  ibadetortu  \
0        2        1      11      1.0      1.0     1.0         1.0   
1        3        1      11      1.0      1.0     1.0         1.0   
2        4        1      11      2.0      1.0     1.0         5.0   
3        4        1      11      NaN      NaN     NaN         NaN   
4        4        1      12      2.0      5.0     5.0         5.0   

   q0050_0016_d  q0050_0005_d  q0050_0001_d  
0           NaN           NaN           NaN  
1           

In [9]:
selected_vars = [
    'q0050_0001', 'q0050_0002', 'q0050_0003', 'q0050_0004', 
    'q0050_0005', 'q0050_0006', 'q0050_0007', 'q0050_0008', 
    'q0050_0009', 'q0050_0010', 'q0050_0011', 'q0050_0012', 
    'q0050_0013', 'q0050_0014', 'q0050_0015', 'q0050_0016', 
    'q0050_0017', 'q0050_0018', 'q0050_0019', 'q0050_0020', 
    'q0050_0021', 'q0050_0022', 'q0050_0023'
]# çalışılacak kimlik değişkenleri

vars_names = {
    'q0050_0001': 'Müslüman',
    'q0050_0002': 'Ateist',
    'q0050_0003': 'İslamcı',
    'q0050_0004': 'Agnostik',
    'q0050_0005': 'Sünni',
    'q0050_0006': 'Deist',
    'q0050_0007': 'Selefi',
    'q0050_0008': 'Milliyetçi',
    'q0050_0009': 'Laik',
    'q0050_0010': 'Solcu',
    'q0050_0011': 'Hümanist',
    'q0050_0012': 'Atatürkçü',
    'q0050_0013': 'Alevi',
    'q0050_0014': 'Şii',
    'q0050_0015': 'Muhafazakar',
    'q0050_0016': 'Dindar',
    'q0050_0017': 'Seküler',
    'q0050_0018': 'Demokrat',
    'q0050_0019': 'Sosyalist',
    'q0050_0020': 'Feminist',
    'q0050_0021': 'LGBT+ hakları savunucusu',
    'q0050_0022': 'Çevreci',
    'q0050_0023': 'Sağcı',
} # adlarını da verelim anlamak kolay olsun



In [12]:
total_sample_size = pol['q0001'].notnull().sum() # sample size'ı missingi olmayan cinsiyet üzerinden bulalım

missing_percentage = (pol[selected_vars].isnull().sum() / total_sample_size) * 100 
# değişkenlerin missinglerini toplam sample size ile oranlayarak yüzde kaç eksik olduğunu görelim

summary = pd.DataFrame({
    "Eksik Yüzde": missing_percentage
}) 

summary.index = [vars_names[col] for col in summary.index] # değişken labellarını isimleriyle değiştirelim

summary = summary.sort_values(by="Eksik Yüzde", ascending=False) # listeyi çoktan aza sıralayalım anlaması kolay olsun

print(summary) # neler varmış bir görelim

                          Eksik Yüzde
Seküler                     78.341969
Selefi                      76.683938
Agnostik                    76.528497
Hümanist                    27.875648
Sosyalist                   26.787565
LGBT+ hakları savunucusu    26.632124
Sağcı                       25.854922
Şii                         25.492228
Deist                       23.989637
Feminist                    23.886010
Sünni                       23.886010
Demokrat                    23.367876
Solcu                       23.264249
Alevi                       19.740933
Muhafazakar                 19.689119
İslamcı                     15.388601
Laik                        14.974093
Milliyetçi                  14.196891
Çevreci                     13.989637
Atatürkçü                   11.968912
Ateist                      11.450777
Dindar                      11.295337
Müslüman                     5.284974


# Bölüm 2: Lasso Yolunda

In [18]:
missing_counts = pol.isnull().sum().sort_values(ascending=False) # Predictor olacak değişkenlerin NaN sayılarını hesaplayıp, çoktan aza sıralayalım

missing_counts_df = missing_counts.reset_index()
missing_counts_df.columns = ["Variable", "Missing_Count"]

output_path = "C:/Users/belki/OneDrive - marmara.edu.tr/Masaüstü/Doktora/Advanced Data Analysis/missing counts.xlsx"  # Daha rahat görmek için excele çekelim

missing_counts_df.to_excel(output_path, index=False)

In [20]:
predictor_variables = [ # Predictor olmasını planladığımız değişkenleri alalım
    "q0001", "q0002", "q0003", "q0004", "q0006", "q0008", "q0009", 
    "q0010", "q0011", "q0013", "q0014", "q0015", "q0016", "q0017", "q0018", 
    "q0019_0001", "q0019_0002", "q0019_0003", "q0019_0004", "q0019_0005", "q0019_0006", 
    "q0019_0007", "q0019_0008", "q0019_0009", "q0020_0001", "q0020_0002", "q0020_0003", 
    "q0020_0004", "q0020_0005", "q0021_0001", "q0021_0002", "q0021_0003", "q0021_0004", 
    "q0021_0005", "q0022_0001", "q0022_0002", "q0022_0003", "q0022_0004", "q0022_0005", 
    "q0023_0001", "q0023_0002", "q0023_0003", "q0023_0004", "q0024_0001", "q0024_0002", 
    "q0024_0003", "q0024_0004", "q0025_0001", "q0025_0002", "q0025_0003", "q0025_0004", 
    "q0025_0005", "q0026_0001", "q0026_0002", "q0026_0003", "q0026_0004", "q0027_0001", 
    "q0027_0002", "q0027_0003", "q0027_0004", "q0027_0005", "q0027_0006", "q0028_0002", 
    "q0028_0003", "q0028_0004", "q0029_0001", "q0029_0002", "q0029_0003", "q0029_0004", 
    "q0029_0005", "q0029_0006", "q0030_0002", "q0030_0003", "q0030_0004", "q0030_0005", 
    "q0030_0006", "q0030_0007", "q0030_0008", "q0030_0009", "q0030_0010", "q0030_0011", 
    "q0030_0012", "q0031_0001", "q0031_0002", "q0031_0003", "q0031_0004", "q0031_0005", 
    "q0031_0006", "q0031_0007", "q0031_0008", "q0031_0009", "q0031_0010", "q0031_0011", 
    "q0031_0012", "q0031_0013", "q0031_0014", "q0031_0016", "q0031_0017", "q0032_0001", 
    "q0032_0002", "q0032_0003", "q0032_0004", "q0032_0005", "q0032_0006", "q0032_0007", 
    "q0032_0008", "q0032_0009", "q0032_0010", "q0033_0001", "q0033_0002", "q0033_0003", 
    "q0033_0004", "q0033_0005", "q0033_0006", "q0035_0001", "q0035_0002", "q0035_0003", 
    "q0035_0004", "q0035_0005", "q0036_0001", "q0036_0002", "q0036_0003", "q0036_0004", 
    "q0036_0005", "q0036_0007", "q0036_0008", "q0036_0009", "q0036_0010", "q0036_0014", 
    "q0036_0015", "q0037_0006", "q0037_0007", "q0038_0001", "q0038_0002", "q0038_0003", 
    "q0038_0004", "q0038_0005", "q0039_0001", "q0039_0002", "q0039_0003", "q0039_0004", 
    "q0039_0005", "q0039_0006", "q0041_0001", "q0041_0002", "q0041_0003", "q0047_0001", 
    "q0047_0002", "q0048_0001", "q0048_0002", "q0048_0003", "q0048_0004", "q0048_0005", 
    "q0048_0006", "q0048_0007", "q0048_0008", "q0048_0009", "q0048_0010", "q0048_0011", 
    "q0049","q0052", "q0053_0002", "q0053_0003", "q0053_0004", "q0053_0005",  "q0053_0008", 
    "q0053_0009", "q0050_0002", "q0050_0003", "q0050_0008","q0050_0009"
]
target_variable = "q0050_0001" # Belirlediğimiz target değişkenimiz

filtered_data = pol[predictor_variables + [target_variable]].dropna() 
# Predictor ve target değerlerinde NaN'leri çıkarıp eşit sayıda gözlemi olan datayı oluşturalım

sample_size = len(filtered_data)
print(f"Toplam örneklem sayısı: {sample_size}") # Bakalım yeni seni sample size'ımız kaç olmuş

Toplam örneklem sayısı: 884


# Bölüm 3: Lasso Denemeleri

In [23]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

In [24]:
X = filtered_data[predictor_variables]  # Predictor değişkenleri tanımlayalım
y = filtered_data[target_variable]  # Target değişkeni tanımlayalım

scaler = StandardScaler() # X'i standardize edelim
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42) 
# Test ve train olarak datayı bölelim, %30'a %70 ayırdık

lasso = Lasso(alpha=0.1)  # Alpha 0.1 cezalandırma katsayısı kullandık
lasso.fit(X_train, y_train)
lasso_pred = lasso.predict(X_test) # ve lasso zamanı
lasso.score(X_test, y_test) # sonuçlar gelsin

0.5821843803795151

In [27]:
mse = mean_squared_error(y_test, lasso_pred)
print(f"MSE: {mse}") # bir de mean standard error'u test edelim

MSE: 0.595958273426675


In [29]:
lasso_coefficients = pd.Series(lasso.coef_, index=predictor_variables) 
# Lasso sonucunda katsayıları görelim, hangilerini predictor seçeceğimize karar vereceğiz
print("\nLasso Katsayıları")
print(lasso_coefficients)

selected_predictors = lasso_coefficients[lasso_coefficients != 0].index.tolist() #Katsayısı 0 olmayanlar bize lazım, onları bir seçelim
print(f"\nLASSO ile seçilen değişkenler: {selected_predictors}") # bir de yazdırıp görelim


Lasso Katsayıları
q0001         0.000000
q0002         0.000000
q0003         0.000000
q0004         0.000000
q0006         0.000000
                ...   
q0053_0009    0.000000
q0050_0002   -0.089904
q0050_0003    0.093233
q0050_0008    0.000000
q0050_0009    0.000000
Length: 171, dtype: float64

LASSO ile seçilen değişkenler: ['q0020_0002', 'q0020_0005', 'q0021_0005', 'q0023_0001', 'q0026_0004', 'q0031_0016', 'q0035_0003', 'q0037_0006', 'q0039_0001', 'q0039_0003', 'q0039_0004', 'q0039_0005', 'q0039_0006', 'q0047_0001', 'q0052', 'q0050_0002', 'q0050_0003']


In [31]:
# Bir de crossvalidation yapıp emin olalım
X = filtered_data[selected_predictors]  # Predictor değişkenler
y = filtered_data[target_variable]  # Target değişken

scaler = StandardScaler() #standardizasyon
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42) #Train test ayrımı, yine 70'e 30 bölelim

lasso_cv = LassoCV(cv=5, random_state=42)  # 5'e bölerek crossvalidation
lasso_cv.fit(X_train, y_train)

lasso = Lasso(alpha=0.001, random_state=42)

cv_scores = cross_val_score(lasso, X_scaled, y, cv=5)

print(f"R²:", cv_scores) #çok fark var mı görelim


R²: [0.61558175 0.65480662 0.49146025 0.61511145 0.5914705 ]


# Bölüm 4: Yeri Dolar Mı?

In [34]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout
from keras.optimizers import Adam
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from keras.models import Model
from keras.layers import Input, Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [56]:
# Multilayer Perceptron (MLP) Denemesi - adam optimizer/mse loss
X = pol[selected_predictors]
y = pol[target_variable] # X ve Y'ler belli olsun

X_missing = X[y.isnull()]
X_complete = X[~y.isnull()]
y_complete = y[~y.isnull()]  # Missing olanları ve olmayanları ayıralım

y_categorical = to_categorical(y_complete -1, num_classes=5)  # Target'ı kategorik hale getirelim

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_complete) # X standardizasyonu

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_categorical, test_size=0.3, random_state=42)

model = Sequential([
    Dense(256, activation="relu", input_dim=X_train.shape[1]),
    Dense(128, activation="relu"),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
    Dense(5, activation="softmax")  # 5 kategori Likert olduğu için çıktımız 5'li
])

model.compile(optimizer="adam", loss="mse", metrics=["mae"])

early_stopping_monitor = EarlyStopping(patience=10)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=16, callbacks=[early_stopping_monitor])

y_pred = model.predict(X_missing)
y_missing_filled = y_pred.argmax(axis=1) + 1  # Kategorileri eski haline getirelim
pol_new = pol.copy()

pol_new.loc[y.isnull(), target_variable] = y_missing_filled
y_pred_complete_1 = model.predict(X_complete).argmax(axis=1) + 1 
y_complete_original_1 = y_complete.values  



C:\Users\belki\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - loss: 0.1465 - mae: 0.3005 - val_loss: 0.1076 - val_mae: 0.1779
Epoch 2/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0974 - mae: 0.1850 - val_loss: 0.1039 - val_mae: 0.1945
Epoch 3/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0875 - mae: 0.1758 - val_loss: 0.1027 - val_mae: 0.2019
Epoch 4/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0948 - mae: 0.1906 - val_loss: 0.1023 - val_mae: 0.2017
Epoch 5/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.1011 - mae: 0.2031 - val_loss: 0.1042 - val_mae: 0.1837
Epoch 6/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0950 - mae: 0.1833 - val_loss: 0.1021 - val_mae: 0.1991
Epoch 7/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0959 - mae: 0.1924 - val_loss: 0.1021 - val_mae: 0.1982
Epoch 8/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0923 - mae: 0.1839 - val_loss: 0.1019 - val_mae: 0.2020
Epoch 9/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0

In [58]:
classification_report_1 = classification_report(y_complete_original_1, y_pred_complete_1)
print(classification_report_1)

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00        97
         2.0       0.00      0.00      0.00        60
         3.0       0.00      0.00      0.00       204
         4.0       0.00      0.00      0.00       186
         5.0       0.70      1.00      0.82      1281

    accuracy                           0.70      1828
   macro avg       0.14      0.20      0.16      1828
weighted avg       0.49      0.70      0.58      1828



C:\Users\belki\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\belki\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\belki\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [44]:
y_missing_filled

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], dtype=int64)

In [62]:
# Multilayer Perceptron (MLP) Denemesi 2 - RMSprop optimizer/categorical_crossentropy loss
X = pol[selected_predictors]
y = pol[target_variable] # X ve Y'ler belli olsun

X_missing = X[y.isnull()]
X_complete = X[~y.isnull()]
y_complete = y[~y.isnull()]  # Missing olanları ve olmayanları ayıralım

y_categorical = to_categorical(y_complete -1, num_classes=5)  # Target'ı kategorik hale getirelim

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_complete) # X standardizasyonu

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_categorical, test_size=0.3, random_state=42)

model = Sequential([
    Dense(256, activation="relu", input_dim=X_train.shape[1]),
      # Doğrulama tahminleri    Dense(128, activation="relu"),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
    Dense(5, activation="softmax")  # 5 kategori Likert olduğu için çıktımız 5'li
])

model.compile(optimizer="RMSprop", loss="categorical_crossentropy", metrics=["accuracy"])

early_stopping_monitor = EarlyStopping(patience=10)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=16, callbacks=[early_stopping_monitor])

X_missing_scaled = scaler.transform(X_missing)
y_pred_missing = model.predict(X_missing_scaled).argmax(axis=1) + 1

pol_new = pol.copy()
pol_new.loc[y.isnull(), target_variable] = y_missing_filled

y_train_original = y_train.argmax(axis=1) + 1
y_val_original = y_val.argmax(axis=1) + 1

X_train_scaled = scaler.transform(X_train)  # Eğitim seti ölçeklendirme
X_val_scaled = scaler.transform(X_val)  # Doğrulama seti ölçeklendirme

y_pred_train = model.predict(X_train_scaled).argmax(axis=1) + 1  # Eğitim seti tahminleri
y_pred_val = model.predict(X_val_scaled).argmax(axis=1) + 1  # Doğrulama seti tahminleri


C:\Users\belki\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.6690 - loss: 1.4344 - val_accuracy: 0.6776 - val_loss: 1.0715
Epoch 2/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7243 - loss: 0.9673 - val_accuracy: 0.6776 - val_loss: 1.0443
Epoch 3/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6998 - loss: 1.0004 - val_accuracy: 0.6776 - val_loss: 1.0403
Epoch 4/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7106 - loss: 0.9793 - val_accuracy: 0.6776 - val_loss: 1.0382
Epoch 5/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6973 - loss: 0.9990 - val_accuracy: 0.6776 - val_loss: 1.0404
Epoch 6/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6990 - loss: 1.0023 - val_accuracy: 0.6776 - val_loss: 1.0460
Epoch 7/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6926 - loss: 1.0113 - val_accuracy: 0.6776 - val_loss: 1.0388
Epoch 8/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7248 - loss: 0.9508 - val_accuracy: 0.6776 - 

C:\Users\belki\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\belki\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [64]:
classification_report_2 = classification_report(y_val_original, y_pred_val)
print(classification_report_2)

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        29
           2       0.00      0.00      0.00        17
           3       0.00      0.00      0.00        64
           4       0.00      0.00      0.00        67
           5       0.68      1.00      0.81       372

    accuracy                           0.68       549
   macro avg       0.14      0.20      0.16       549
weighted avg       0.46      0.68      0.55       549



C:\Users\belki\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\belki\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\belki\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
y_missing_filled_1

In [70]:
# Autoencoder - adam optimizer/categorical_crossentropy loss
X = pol[selected_predictors]
y = pol[target_variable]

X_missing = X[y.isnull()]
X_complete = X[~y.isnull()]
y_complete = y[~y.isnull()]

y_complete = y_complete - 1  
y_categorical = to_categorical(y_complete, num_classes=5)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_complete)

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_categorical, test_size=0.3, random_state=42)


input_dim = X_train.shape[1]
encoding_dim = 32  # Kodlama boyutu

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="linear")(encoder) # Encoder ve decoder tanımlayalım

autoencoder = Model(inputs=input_layer, outputs=decoder)

autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss="mse") 

autoencoder.fit(X_train, X_train, # Self supervised learning olduğu için böyle
                validation_data=(X_val, X_val), 
                epochs=50, 
                batch_size=16, 
                verbose=1) # Autoencoder eğitim

encoder_model = Model(inputs=input_layer, outputs=encoder)

classifier_input = Input(shape=(encoding_dim,))
classifier_layer = Dense(128, activation="relu")(classifier_input)
classifier_layer = Dense(64, activation="relu")(classifier_layer)
classifier_layer = Dense(32, activation="relu")(classifier_layer)
classifier_layer = Dense(16, activation="relu")(classifier_layer)
classifier_output = Dense(5, activation="softmax")(classifier_layer) # Ordinal sınıflandırma modeli

classifier_model = Model(inputs=classifier_input, outputs=classifier_output)

classifier_model.compile(optimizer=Adam(learning_rate=0.001), 
                         loss="categorical_crossentropy", 
                         metrics=["accuracy"]) 

X_encoded = encoder_model.predict(X_scaled)

early_stopping_monitor = EarlyStopping(patience=10)
classifier_model.fit(X_encoded, y_categorical,
                     callbacks=[early_stopping_monitor],
                     validation_split=0.3, 
                     epochs=100, 
                     batch_size=16, 
                     verbose=1)

X_missing_scaled = scaler.transform(X_missing)
y_pred_missing = model.predict(X_missing_scaled).argmax(axis=1) + 1

pol_new = pol.copy()
pol_new.loc[y.isnull(), target_variable] = y_missing_filled


y_train_original_2 = y_train.argmax(axis=1) + 1
y_val_original_2 = y_val.argmax(axis=1) + 1

X_train_scaled_2 = scaler.transform(X_train)  # Eğitim seti ölçeklendirme
X_val_scaled_2 = scaler.transform(X_val)  # Doğrulama seti ölçeklendirme

y_pred_train_2 = model.predict(X_train_scaled).argmax(axis=1) + 1  # Eğitim seti tahminleri
y_pred_val_2 = model.predict(X_val_scaled).argmax(axis=1) + 1  # Doğrulama seti tahminleri


Epoch 1/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - loss: nan - val_loss: nan
Epoch 2/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: nan - val_loss: nan
Epoch 3/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: nan - val_loss: nan
Epoch 4/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: nan - val_loss: nan
Epoch 5/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: nan - val_loss: nan
Epoch 6/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: nan - val_loss: nan
Epoch 7/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: nan - val_loss: nan
Epoch 8/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: nan - val_loss: nan
Epoch 9/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: nan - val_loss: nan
Epoch 10/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: nan - val_loss: nan
Epoch 11/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: nan - val_loss: nan
Epoch 12/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: nan - val_loss: nan
Epoch 13/50
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: n

C:\Users\belki\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\belki\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [72]:
classification_report_3 = classification_report(y_val_original_2, y_pred_val_2)
print(classification_report_3)

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        29
           2       0.00      0.00      0.00        17
           3       0.00      0.00      0.00        64
           4       0.00      0.00      0.00        67
           5       0.68      1.00      0.81       372

    accuracy                           0.68       549
   macro avg       0.14      0.20      0.16       549
weighted avg       0.46      0.68      0.55       549



C:\Users\belki\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\belki\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\belki\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [116]:
y_missing_filled_2

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], dtype=int64)

# Yeniden Deneyelim, Bu Kez Dikotomik

# Bölüm 1: Bazı Kararlar

In [86]:
import openpyxl
import pandas as pd

In [88]:
file_path = "C:/Users/belki/OneDrive - marmara.edu.tr/Masaüstü/Yüksek Lisans/Tez/Affective Polarization/TIDA Python.xlsx" #excel dosyası

pol = pd.read_excel(file_path)

print(pol.head()) #veri kontrolü

   q0001  q0002  q0003  q0004  q0005  q0006  q0007  q0008  q0009  q0010  ...  \
0      2      4     73      1    NaN      1    5.0      5      3      3  ...   
1      2      9     73      1    NaN      2    NaN      5      8      9  ...   
2      2      8     73      1    NaN      2    NaN      7      1      3  ...   
3      1     29     73      2    1.0      2    NaN      7      6      6  ...   
4      2     20     40      2    4.0      2    NaN      5      2      2  ...   

   dindarR  kirkent  nutstr  basortu  disortu  evortu  ibadetortu  \
0        2        1      11      1.0      1.0     1.0         1.0   
1        3        1      11      1.0      1.0     1.0         1.0   
2        4        1      11      2.0      1.0     1.0         5.0   
3        4        1      11      NaN      NaN     NaN         NaN   
4        4        1      12      2.0      5.0     5.0         5.0   

   q0050_0016_d  q0050_0005_d  q0050_0001_d  
0           NaN           NaN           NaN  
1           

In [89]:
selected_vars = [
    'q0050_0001', 'q0050_0002', 'q0050_0003', 'q0050_0004', 
    'q0050_0005', 'q0050_0006', 'q0050_0007', 'q0050_0008', 
    'q0050_0009', 'q0050_0010', 'q0050_0011', 'q0050_0012', 
    'q0050_0013', 'q0050_0014', 'q0050_0015', 'q0050_0016', 
    'q0050_0017', 'q0050_0018', 'q0050_0019', 'q0050_0020', 
    'q0050_0021', 'q0050_0022', 'q0050_0023', 'q0050_0005_d',
    'q0050_0016_d', 'q0050_0001_d'
]# çalışılacak kimlik değişkenleri

vars_names = {
    'q0050_0001': 'Müslüman',
    'q0050_0002': 'Ateist',
    'q0050_0003': 'İslamcı',
    'q0050_0004': 'Agnostik',
    'q0050_0005': 'Sünni',
    'q0050_0006': 'Deist',
    'q0050_0007': 'Selefi',
    'q0050_0008': 'Milliyetçi',
    'q0050_0009': 'Laik',
    'q0050_0010': 'Solcu',
    'q0050_0011': 'Hümanist',
    'q0050_0012': 'Atatürkçü',
    'q0050_0013': 'Alevi',
    'q0050_0014': 'Şii',
    'q0050_0015': 'Muhafazakar',
    'q0050_0016': 'Dindar',
    'q0050_0017': 'Seküler',
    'q0050_0018': 'Demokrat',
    'q0050_0019': 'Sosyalist',
    'q0050_0020': 'Feminist',
    'q0050_0021': 'LGBT+ hakları savunucusu',
    'q0050_0022': 'Çevreci',
    'q0050_0023': 'Sağcı',
    'q0050_0005_d': 'Sünni dikotomik',
    'q0050_0001_d': 'Müslüman dikotomik',
    'q0050_0016_d': 'Dindar dikotomik',
} # adlarını da verelim anlamak kolay olsun


In [91]:
total_sample_size = pol['q0001'].notnull().sum() # sample size'ı missingi olmayan cinsiyet üzerinden bulalım

missing_percentage = (pol[selected_vars].isnull().sum() / total_sample_size) * 100 
# değişkenlerin missinglerini toplam sample size ile oranlayarak yüzde kaç eksik olduğunu görelim

summary = pd.DataFrame({
    "Eksik Yüzde": missing_percentage
}) 

summary.index = [vars_names[col] for col in summary.index] # değişken labellarını isimleriyle değiştirelim

summary = summary.sort_values(by="Eksik Yüzde", ascending=False) # listeyi çoktan aza sıralayalım anlaması kolay olsun

print(summary) # neler varmış bir görelim

                          Eksik Yüzde
Seküler                     78.341969
Selefi                      76.683938
Agnostik                    76.528497
Hümanist                    27.875648
Sosyalist                   26.787565
LGBT+ hakları savunucusu    26.632124
Sağcı                       25.854922
Şii                         25.492228
Deist                       23.989637
Feminist                    23.886010
Sünni dikotomik             23.886010
Sünni                       23.886010
Demokrat                    23.367876
Solcu                       23.264249
Alevi                       19.740933
Muhafazakar                 19.689119
İslamcı                     15.388601
Laik                        14.974093
Milliyetçi                  14.196891
Çevreci                     13.989637
Atatürkçü                   11.968912
Ateist                      11.450777
Dindar dikotomik            11.295337
Dindar                      11.295337
Müslüman                     5.284974
Müslüman dik

# Bölüm 2: Lasso Yolunda

In [95]:
missing_counts = pol.isnull().sum().sort_values(ascending=False) # Predictor olacak değişkenlerin NaN sayılarını hesaplayıp, çoktan aza sıralayalım

missing_counts_df = missing_counts.reset_index()
missing_counts_df.columns = ["Variable", "Missing_Count"]

output_path = "C:/Users/belki/OneDrive - marmara.edu.tr/Masaüstü/Doktora/Advanced Data Analysis/missing counts.xlsx"  # Daha rahat görmek için excele çekelim

missing_counts_df.to_excel(output_path, index=False)

In [98]:
predictor_variables = [ # Predictor olmasını planladığımız değişkenleri alalım
     "q0001", "q0002", "q0003", "q0004", "q0006", "q0008", "q0009", 
    "q0010", "q0011", "q0013", "q0014", "q0015", "q0016", "q0017", "q0018", 
    "q0019_0001", "q0019_0002", "q0019_0003", "q0019_0004", "q0019_0005", "q0019_0006", 
    "q0019_0007", "q0019_0008", "q0019_0009", "q0020_0001", "q0020_0002", "q0020_0003", 
    "q0020_0004", "q0020_0005", "q0021_0001", "q0021_0002", "q0021_0003", "q0021_0004", 
    "q0021_0005", "q0022_0001", "q0022_0002", "q0022_0003", "q0022_0004", "q0022_0005", 
    "q0023_0001", "q0023_0002", "q0023_0003", "q0023_0004", "q0024_0001", "q0024_0002", 
    "q0024_0003", "q0024_0004", "q0025_0001", "q0025_0002", "q0025_0003", "q0025_0004", 
    "q0025_0005", "q0026_0001", "q0026_0002", "q0026_0003", "q0026_0004", "q0027_0001", 
    "q0027_0002", "q0027_0003", "q0027_0004", "q0027_0005", "q0027_0006", "q0028_0002", 
    "q0028_0003", "q0028_0004", "q0029_0001", "q0029_0002", "q0029_0003", "q0029_0004", 
    "q0029_0005", "q0029_0006", "q0030_0002", "q0030_0003", "q0030_0004", "q0030_0005", 
    "q0030_0006", "q0030_0007", "q0030_0008", "q0030_0009", "q0030_0010", "q0030_0011", 
    "q0030_0012", "q0031_0001", "q0031_0002", "q0031_0003", "q0031_0004", "q0031_0005", 
    "q0031_0006", "q0031_0007", "q0031_0008", "q0031_0009", "q0031_0010", "q0031_0011", 
    "q0031_0012", "q0031_0013", "q0031_0014", "q0031_0016", "q0031_0017", "q0032_0001", 
    "q0032_0002", "q0032_0003", "q0032_0004", "q0032_0005", "q0032_0006", "q0032_0007", 
    "q0032_0008", "q0032_0009", "q0032_0010", "q0033_0001", "q0033_0002", "q0033_0003", 
    "q0033_0004", "q0033_0005", "q0033_0006", "q0035_0001", "q0035_0002", "q0035_0003", 
    "q0035_0004", "q0035_0005", "q0036_0001", "q0036_0002", "q0036_0003", "q0036_0004", 
    "q0036_0005", "q0036_0007", "q0036_0008", "q0036_0009", "q0036_0010", "q0036_0014", 
    "q0036_0015", "q0037_0006", "q0037_0007", "q0038_0001", "q0038_0002", "q0038_0003", 
    "q0038_0004", "q0038_0005", "q0039_0001", "q0039_0002", "q0039_0003", "q0039_0004", 
    "q0039_0005", "q0039_0006", "q0041_0001", "q0041_0002", "q0041_0003", "q0047_0001", 
    "q0047_0002", "q0048_0001", "q0048_0002", "q0048_0003", "q0048_0004", "q0048_0005", 
    "q0048_0006", "q0048_0007", "q0048_0008", "q0048_0009", "q0048_0010", "q0048_0011", 
    "q0049","q0052"  
   
]
target_variable = "q0050_0001_d" # Belirlediğimiz target değişkenimiz

filtered_data = pol[predictor_variables + [target_variable]].dropna() # Predictor ve target değerlerinde NaN'leri çıkarıp eşit sayıda gözlemi olan datayı oluşturalım

sample_size = len(filtered_data)
print(f"Toplam örneklem sayısı: {sample_size}") # Bakalım yeni seni sample size'ımız kaç olmuş

Toplam örneklem sayısı: 1129


# Bölüm 3: Lasso Denemeleri

In [103]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [106]:
X = filtered_data[predictor_variables]  # Predictor değişkenleri tanımlayalım
y = filtered_data[target_variable]  # Target değişkeni tanımlayalım

scaler = StandardScaler() # X'i standardize edelim
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42) 
# Test ve train olarak datayı bölelim, %30'a %70 ayırdık

lasso = Lasso(alpha=0.01)  # Alpha 0.01 cezalandırma katsayısı kullandık
lasso.fit(X_train, y_train)
lasso_pred = lasso.predict(X_test) # ve lasso zamanı
lasso.score(X_test, y_test) # sonuçlar gelsin

0.4292082600515461

In [109]:
mse = mean_squared_error(y_test, lasso_pred)
print(f"MSE: {mse}") # bir de mean standard error'u test edelim

MSE: 0.024138737534040645


In [112]:
lasso_coefficients = pd.Series(lasso.coef_, index=predictor_variables) 
# Lasso sonucunda katsayıları görelim, hangilerini predictor seçeceğimize karar vereceğiz
print("\nLasso Katsayıları")
print(lasso_coefficients)

selected_predictors = lasso_coefficients[lasso_coefficients != 0].index.tolist() #Katsayısı 0 olmayanlar bize lasım, onları bir seçelim
print(f"\nLASSO ile seçilen değişkenler: {selected_predictors}") # bir de yazdırıp görelim


Lasso Katsayıları
q0001        -0.000000
q0002        -0.000000
q0003         0.000000
q0004        -0.000000
q0006        -0.000000
                ...   
q0048_0009    0.000196
q0048_0010   -0.000000
q0048_0011    0.000000
q0049        -0.000000
q0052        -0.027199
Length: 161, dtype: float64

LASSO ile seçilen değişkenler: ['q0020_0001', 'q0021_0004', 'q0022_0001', 'q0022_0004', 'q0023_0001', 'q0025_0003', 'q0026_0001', 'q0027_0001', 'q0029_0006', 'q0030_0009', 'q0031_0005', 'q0032_0002', 'q0032_0007', 'q0033_0002', 'q0036_0007', 'q0037_0006', 'q0038_0004', 'q0039_0001', 'q0039_0004', 'q0039_0005', 'q0039_0006', 'q0048_0009', 'q0052']


In [115]:
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

In [118]:
# Bir de crossvalidation yapıp emin olalım
X = filtered_data[predictor_variables]  # Predictor değişkenler
y = filtered_data[target_variable]  # Target değişken

scaler = StandardScaler() #standardizasyon
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42) #Train test ayrımı, yine 30'a 70 bölelim

lasso_cv = LassoCV(cv=5, random_state=42)  # 5'e bölerek crossvalidation
lasso_cv.fit(X_train, y_train)

lasso = Lasso(alpha=0.01, random_state=42)

cv_scores = cross_val_score(lasso, X_scaled, y, cv=5)

print(f"R²:", cv_scores) #çok fark var mı görelim

R²: [0.31963566 0.44910642 0.50666744 0.21022696 0.57008838]


# Bölüm 4: Yeri Dolar Mı?

In [124]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from sklearn.metrics import classification_report

In [127]:
# Binary classification multilayer perceptron Denemesi - adam optimizer/binary_crossentropy loss
X = pol[selected_predictors] # X ve y tanımlayalım
y = pol[target_variable]

X_missing = X[y.isnull()]
X_complete = X[~y.isnull()]
y_complete = y[~y.isnull()] # Missinglerle missing olmayanları ayıralım
y_complete = y_complete - 1 
y_complete = to_categorical(y_complete)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_complete) #optimizasyon

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_complete, test_size=0.3, random_state=42)

model = Sequential([
    Dense(256, activation='relu', input_dim=X_train.shape[1]),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'), 
    Dense(16, activation='relu'), 
    Dense(8, activation='relu'),
    Dense(2, activation='sigmoid')  # Binary olduğu için sigmoid
])

model.compile(optimizer=Adam(learning_rate=0.01),  
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.fit(X_scaled, y_complete, 
                    validation_split=0.3, 
                    epochs=100, 
                    batch_size=16, 
                    verbose=1)

y_pred = model.predict(X_complete)
y_pred_classes = np.argmax(y_pred, axis=1)  
y_true_classes = np.argmax(y_complete, axis=1)

C:\Users\belki\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.9244 - loss: 0.3209 - val_accuracy: 0.9490 - val_loss: 0.2030
Epoch 2/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9408 - loss: 0.2419 - val_accuracy: 0.9490 - val_loss: 0.2187
Epoch 3/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9538 - loss: 0.1950 - val_accuracy: 0.9490 - val_loss: 0.2021
Epoch 4/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9488 - loss: 0.2054 - val_accuracy: 0.9490 - val_loss: 0.2036
Epoch 5/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9482 - loss: 0.2073 - val_accuracy: 0.9490 - val_loss: 0.2017
Epoch 6/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9605 - loss: 0.1747 - val_accuracy: 0.9490 - val_loss: 0.2189
Epoch 7/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9477 - loss: 0.2133 - val_accuracy: 0.9490 - val_loss: 0.2179
Epoch 8/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9353 - loss: 0.2441 - val_accuracy: 0.9490 

In [129]:
print(classification_report(y_true_classes, y_pred_classes))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        97
           1       0.95      1.00      0.97      1731

    accuracy                           0.95      1828
   macro avg       0.47      0.50      0.49      1828
weighted avg       0.90      0.95      0.92      1828



C:\Users\belki\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\belki\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\belki\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [131]:
# Binary classification multilayer perceptron (MLP) Denemesi 2 - adam optimizer/mse loss
X_missing = X[y.isnull()]
X_complete = X[~y.isnull()]
y_complete = y[~y.isnull()]
y_complete = y_complete - 1  # 1 ve 2 sınıflarını 0 ve 1'e dönüştürelim
y_complete = to_categorical(y_complete)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_complete) #optimizasyon

model = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),     
    Dense(32, activation='relu'),  
    Dense(16, activation='relu'),
    Dense(8, activation='relu'), 
    Dense(2, activation='softmax')  #kategorik olduğunu varsayarak softmax     
])


model.compile(optimizer="adam", 
              loss="mse", 
              metrics=["accuracy"])

early_stopping_monitor = EarlyStopping(patience=10)
model.fit(X_scaled, y_complete,
                    callbacks=[early_stopping_monitor],
                    validation_split=0.2, 
                    epochs=100, 
                    batch_size=32, 
                    verbose=1)

y_pred_2 = model.predict(X_complete)


C:\Users\belki\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 8s 29ms/step - accuracy: 0.8798 - loss: 0.2375 - val_accuracy: 0.9262 - val_loss: 0.1783
Epoch 2/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9423 - loss: 0.1498 - val_accuracy: 0.9262 - val_loss: 0.0909
Epoch 3/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9546 - loss: 0.0640 - val_accuracy: 0.9262 - val_loss: 0.0693
Epoch 4/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9524 - loss: 0.0476 - val_accuracy: 0.9262 - val_loss: 0.0683
Epoch 5/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9567 - loss: 0.0421 - val_accuracy: 0.9262 - val_loss: 0.0685
Epoch 6/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9574 - loss: 0.0411 - val_accuracy: 0.9262 - val_loss: 0.0687
Epoch 7/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9566 - loss: 0.0416 - val_accuracy: 0.9262 - val_loss: 0.0688
Epoch 8/100
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9547 - loss: 0.0433 - val_accuracy: 0.926

In [133]:
y_pred_labels = np.argmax(y_pred_2, axis=1)

y_true_labels = np.argmax(y_complete, axis=1)

print(classification_report(y_true_labels, y_pred_labels))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        97
           1       0.95      1.00      0.97      1731

    accuracy                           0.95      1828
   macro avg       0.47      0.50      0.49      1828
weighted avg       0.90      0.95      0.92      1828



C:\Users\belki\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\belki\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\belki\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Bölüm 5: Neler Var?

- DNN (Deep Neural Network)
- ABBN (Approximate Bayesian Bootstrap)